<a href="https://colab.research.google.com/github/seongyongju/SAFE_AI_Project_Proposal/blob/main/(CVPR)Basic_CNN_Tuning_Ver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import torch
import torch.nn as nn
import numpy as np
from torchvision import models, transforms, datasets
from torch.autograd import Variable
import torch.nn.functional as F
from torch.utils.data import DataLoader

import matplotlib.pyplot as plt
import cv2
from google.colab import files

!pip install pdf2image
!apt-get install -y poppler-utils
from pdf2image import convert_from_path
from PIL import Image


In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

train_path = "/content/drive/MyDrive/SAFE_AI(Project_proposal)/CVPR_Dataset_LowRes/train"
test_path = "/content/drive/MyDrive/SAFE_AI(Project_proposal)/CVPR_Dataset_LowRes/test"

train_dataset = datasets.ImageFolder(train_path, transform=transform)
test_dataset = datasets.ImageFolder(test_path, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

In [ ]:
class CNN_FT(nn.Module):
    def __init__(self, p_dropout=0.3):
        super().__init__()

        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.bn1   = nn.BatchNorm2d(16)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.bn2   = nn.BatchNorm2d(32)

        self.flat  = nn.Flatten()
        self.fc1   = nn.Linear(32*56*56, 128)
        self.dp1   = nn.Dropout(p_dropout)
        self.fc2   = nn.Linear(128, 2)

        self.apply(self._init_weights)

    def _init_weights(self, m):
        """Kaiming (He) initialization for ReLU 계열"""
        if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
            nn.init.kaiming_normal_(m.weight, nonlinearity='relu')
            if m.bias is not None:
                nn.init.constant_(m.bias, 0)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.max_pool2d(x, 2)

        x = self.flat(x)
        x = F.relu(self.fc1(x))
        x = self.dp1(x)
        x = self.fc2(x)
        return x

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNN_FT().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_acc = 100 * correct / total
    avg_loss = running_loss / len(train_loader)
    print(f"[Epoch {epoch+1}] 🔥 Loss: {avg_loss:.4f} | Accuracy: {train_acc:.2f}%")

In [ ]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_acc = 100 * correct / total
print(f"테스트 정확도: {test_acc:.2f}%")